# 3.3.3.2 Realistic simulation setup
Obtain the centroid for each chessboard 

In [1]:
vscode = 1


In [2]:
if(vscode == 0):
    # for vscode
    # for jupyter notebook
    from mpl_toolkits.mplot3d import axes3d
    import matplotlib.pyplot as plt

    %matplotlib notebook
elif(vscode == 1):
    %matplotlib qt
    

In [3]:
import matplotlib.pyplot as plt
import numpy as np
import cv2
 
from typing import Sequence
from calib_lib import *
from matplotlib.collections import PatchCollection      # pathcoletion for multiple patches
import matplotlib.cm as cm                              # for colormaps
from matplotlib.patches import Rectangle                # for rectangle representation
from matplotlib.patches import Ellipse                  # for ellipse representation   
from pixelization_lib import *   
#from ellipse import LsqEllipse                          # pip install lsq-ellipse


DECIMALS = 2            # how many decimal places to use in print


In [11]:
import logging 
import numpy as np
from skimage.measure import EllipseModel
from matplotlib.patches import Ellipse
import matplotlib.pyplot as plt

In [12]:
F = 16                                       # focal length( in mm )
gridSize = 1024
subpixel_grid = 128
image_size = np.array([1936,1216])               # sensor size(in pixels)
#image_size = np.array([11.345,7.126])
pixel_width = 5.86e-3                       # pixel size in mm
PX= image_size[0]/2.0                       # principal point_i x-coordinate
PY= image_size[1]/2.0                       # principal point_i y-coordinate
IMAGE_HEIGTH = image_size[1]
IMAGE_WIDTH = image_size[0]
THETA_X = 0                                 # roll angle
THETA_Y = 0                                 # pitch angle
THETA_Z = 0                                 # yaw angle

# camera Right
THETA_X_R = 0                                 # roll angle
THETA_Y_R = 0                                 # pitch angle
THETA_Z_R= 0                                 # yaw angle
# camera Left
THETA_X_L = 0                                 # roll angle
THETA_Y_L = 0                                 # pitch angle
THETA_Z_L= 0                                 # yaw angle

C_L = np.array([0,0,0])                     # camera centre
C_R = np.array([500,0,0])



In [13]:
# from 1 meter to 300 meters in steps of 10 meters
zdistance = np.arange(0,320000,1000)
zdistance[0] = 1000
xdistance = np.ones((zdistance.shape[0],))*250
ydistance = np.ones((zdistance.shape[0],))*10
X =np.array([xdistance,ydistance,zdistance]).T


x_zhang_L,x_zhang_R,X_arr_L,X_arr_R,x0_x,x0_y= create_1024_circle(X,PX,PY,thetax=THETA_X_L,thetay = THETA_Y_L,thetaz = THETA_Z_L,trans_x= -C_L[0],trans_y= -C_L[1],trans_z= -C_L[2],F = F,mx = (1/pixel_width),my =(1/pixel_width), npoints = 1024,radius = 10)

In [14]:
x0_L = x_zhang_L[:,:,0]
y0_L = x_zhang_L[:,:,1]


x0_R = x_zhang_R[:,:,0]
y0_R = x_zhang_R[:,:,1]

In [16]:
xy_L_arr = np.load("xy_centroid_estimation_L.npy")
xy_R_arr = np.load("xy_centroid_estimation_R.npy")
ab_L_arr = np.load("ab_L_arr.npy")
ab_R_arr = np.load("ab_R_arr.npy") 
theta_arr_L = np.load("theta_arr_L.npy")
theta_arr_R = np.load("theta_arr_R.npy")


In [17]:
def spot_center_coord(cx0,cy0,pixelGridX,pixelGridY):
    cx0_max = np.where((cx0 < pixelGridX))
    cx0_min = np.where((cx0 > pixelGridX))
    #print(cx0_max[0][0])
    #print(cx0_min[0][-1])
    cy0_max = np.where((cy0 < pixelGridY))
    cy0_min = np.where((cy0 > pixelGridY))
    #print(cy0_max[0][0])
    #print(cy0_min[0][-1])

    return cx0_max[0][0],cx0_min[0][-1],cy0_max[0][0],cy0_min[0][-1]


def findcircle(x_arr):
    """
    Based on the image coordinates, and knowing it forms a square/rectangle, finds the centroid and radius

    Parameters
    ----------
    x_arr:  np.array
        image coordinates
    Returns
    -------
    cx0:    np.float
        centroid x
    cy0:    np.float
        centroid y    
    r0 : np.float
        circle radius
    """  
    
    Ax  = x_arr[0,0]
    Ay  = x_arr[0,1]
    Bx  = x_arr[int(x_arr.shape[0]/4),0]
    By  = x_arr[int(x_arr.shape[0]/4),1]
    Cx  = x_arr[int(x_arr.shape[0]/2),0]
    Cy  = x_arr[int(x_arr.shape[0]/2),1]
    Dx  = x_arr[int((x_arr.shape[0]/4*3)-1),0]
    Dy  = x_arr[int((x_arr.shape[0]/4*3)-1),1]


    cx0 = np.sum(x_arr[:,0])/np.sum(x_arr[:,0].shape[0])
    cy0 = np.sum(x_arr[:,1])/np.sum(x_arr[:,1].shape[0])

    r0 = np.abs((Dy-By)/2)
    
    return cx0,cy0,r0

def createcircle(x_arr,nrpoints = 100):
    """
    create the circle, using the function findcircle()

    Parameters
    ----------
    x_arr:  np.array
        image coordinates
    nrpoints: integer
        number of points that compose the circle
    Returns
    -------
    circle_x:    np.array
        points that define the circle in the cx0 coordinate
    circle_y:    np.array
        points that define the circle in the cy0 coordinate
    cx0:    np.float
        centroid x
    cy0:    np.float
        centroid y    
    r0 : np.float
        circle radius    
    """    

    cx0,cy0,r0 = findcircle(x_arr)

    theta = np.linspace(0, 2*np.pi, nrpoints)
    x = r0*np.cos(theta)
    y = r0*np.sin(theta)

    circle_x = x+cx0
    circle_y = y+cy0


    return circle_x, circle_y,cx0,cy0,r0

In [18]:
"""
NMI : 2021-11-11 
Minimum Volume Enclosing Ellipsoids, see e.g.
NIMA MOSHTAGH : MINIMUM VOLUME ENCLOSING ELLIPSOIDS
or 
Linus Källberg : Minimum_Enclosing_Balls_and_Ellipsoids (Thesis)
"""
from warnings import warn

from numpy import pi
from numpy import sqrt
from numpy import arccos
from numpy import dot, outer
from numpy import diag, transpose
from numpy import append
from numpy import asarray
from numpy import ones
from numpy import argmax

from numpy.linalg import inv
from numpy.linalg import norm
from numpy.linalg import eig


def mvee( data, tolerance=1e-4, maxcnt=10000 ):
    """
    param data: list of xy data points
    param tolerance: termination condition for iterative approximation
    param maxcnt: maximum number of iterations
    type data: iterable of float
    type tolerance: float
    return: (offset, semiaxis, angle)
    return type: ( (float, float), (float, float), float )
    """
    locdata = asarray( data )
    N = len( locdata )
    if not locdata.shape == ( N, 2):
        raise ValueError ( " data must be of shape( n, 2 )" )
    if tolerance >= 1 or tolerance <= 0:
        raise ValueError (" 0 < tolerance < 1 required")
    if not isinstance( maxcnt, int ):
        raise TypeError
    if not maxcnt > 0:
        raise ValueError
    count = 1
    err = 1
    d = 2
    d1 = d + 1
    u = ones( N ) / N
    P = transpose( locdata )
    Q = append( P, ones( N ) ).reshape( 3, -1 )
    while ( err > tolerance):
        X = dot( Q, dot( diag( u ), transpose( Q ) ) )
        M = diag( 
            dot( 
                transpose( Q ),
                dot(
                    inv( X ),
                    Q
                )
            )
        )
        maximum = max( M )
        j = argmax( M )
        step_size = ( maximum - d1 ) / ( d1 * ( maximum - 1 ) )
        new_u = ( 1 - step_size ) * u
        new_u[ j ] += step_size
        err = norm( new_u - u )
        count = count + 1
        u = new_u
        if count > maxcnt:
            warn(
                "Process did not converge in {} steps".format(
                    count - 1
                ),
                UserWarning
            )
            break
    U = diag( u )
    c = dot( P,  u )
    A = inv(
        dot(
            P,
            dot( U, transpose( P ) )
        ) - outer( c, c )
    ) / d
    E, V = eig( A )
    phiopt = arccos( V[ 0, 0 ] )
    if V[ 0, 1 ] < 0: 
        phiopt = 2 * pi - phiopt
    ### cw vs ccw and periodicity of pi
    phiopt = -phiopt % pi
    sol =  (  c, sqrt( 1.0 / E ), phiopt)
    return sol

In [19]:
x0_L = x_zhang_L[:,:,0]
y0_L = x_zhang_L[:,:,1]


x0_R = x_zhang_R[:,:,0]
y0_R = x_zhang_R[:,:,1]

### Find the center of the circle(point)

In [20]:
# Generate pixel grid
pixelGridX  = (np.arange(0,image_size[0])) 
pixelGridY  = (np.arange(0,image_size[1]))
pixelGridXX, pixelGridYY = np.meshgrid(pixelGridX,pixelGridY)

cx0_L = np.zeros((x_zhang_L.shape[0],1))
cy0_L = np.zeros((x_zhang_L.shape[0],1))
r0_L = np.zeros((x_zhang_L.shape[0],1))
cx0_R = np.zeros((x_zhang_R.shape[0],1))
cy0_R = np.zeros((x_zhang_R.shape[0],1))
r0_R = np.zeros((x_zhang_R.shape[0],1))

#
x0_cL = np.zeros((x_zhang_L.shape[0],gridSize))
y0_cL = np.zeros((x_zhang_L.shape[0],gridSize))
x0_cR = np.zeros((x_zhang_R.shape[0],gridSize))
y0_cR = np.zeros((x_zhang_R.shape[0],gridSize))


for point_i in range(x0_L.shape[0]):

    #circle definition 
    # multiply by pixel_width to get the results in mm, not in pixels
    x0_cL[point_i,:],y0_cL[point_i,:],cx0_L[point_i,:],cy0_L[point_i,:],r0_L[point_i,:] = createcircle(x_zhang_L[point_i,:],gridSize)
    x0_cR[point_i,:],y0_cR[point_i,:],cx0_R[point_i,:],cy0_R[point_i,:],r0_R[point_i,:] = createcircle(x_zhang_R[point_i,:],gridSize)

In [22]:
x_L_lst = []
y_L_lst = []

#fig = plt.figure()
#ax = fig.add_subplot()
for point in range(x0_L.shape[0]):
    #ax.plot(x0_L[point,:],y0_L[point,:])
    max_pt_x_L = np.max(x_zhang_L[point,:,0])
    max_pt_y_L = np.max(x_zhang_L[point,:,1])
    min_pt_x_L = np.min(x_zhang_L[point,:,0])
    min_pt_y_L = np.min(x_zhang_L[point,:,1])

    pts = np.array([[max_pt_x_L,max_pt_y_L],[max_pt_x_L,min_pt_y_L],[min_pt_x_L,min_pt_y_L],[min_pt_x_L,max_pt_y_L]])
    cmin_pxl_L = np.zeros((pts.shape[0],pts.shape[1]))
    for pts_i,pts_d in enumerate(pts):
        cx0max_L,cx0min_L,cy0max_L,cy0min_L  = spot_center_coord(pts_d[0],pts_d[1],pixelGridX,pixelGridY)

        # max and min points in (x,y) coordinates
        cmin_pxl_L[pts_i,0] = (cx0min_L)
        cmin_pxl_L[pts_i,1] = (cy0min_L)

    x_L_lst.append(np.arange(cmin_pxl_L[3,0],cmin_pxl_L[0,0]+1,1))
    y_L_lst.append(np.arange(cmin_pxl_L[1,1],cmin_pxl_L[0,1]+1,1))
    xx,yy = np.meshgrid(x_L_lst[point],y_L_lst[point])

#ax.plot(xx,yy, color = 'g',ls = "None", marker = "*")

#ax.set_aspect(1)    
#plt.show()

In [23]:
x_R_lst = []
y_R_lst = []

for point in range(x0_R.shape[0]):
    #ax.plot(x0_L[chess_b_i,point,:],y0_L[chess_b_i,point,:])
    max_pt_x_R = np.max(x_zhang_R[point,:,0])
    max_pt_y_R = np.max(x_zhang_R[point,:,1])
    min_pt_x_R = np.min(x_zhang_R[point,:,0])
    min_pt_y_R = np.min(x_zhang_R[point,:,1])

    pts = np.array([[max_pt_x_R,max_pt_y_R],[max_pt_x_R,min_pt_y_R],[min_pt_x_R,min_pt_y_R],[min_pt_x_R,max_pt_y_R]])
    cmin_pxl_R = np.zeros((pts.shape[0],pts.shape[1]))
    for pts_i,pts_d in enumerate(pts):
        cx0max_R,cx0min_R,cy0max_R,cy0min_R  = spot_center_coord(pts_d[0],pts_d[1],pixelGridX,pixelGridY)

        # max and min points in (x,y) coordinates
        cmin_pxl_R[pts_i,0] = (cx0min_R)
        cmin_pxl_R[pts_i,1] = (cy0min_R)

    x_R_lst.append(np.arange(cmin_pxl_R[3,0],cmin_pxl_R[0,0]+1,1))
    y_R_lst.append(np.arange(cmin_pxl_R[1,1],cmin_pxl_R[0,1]+1,1))
    xx,yy = np.meshgrid(x_R_lst[point],y_R_lst[point])

    #ax.plot(xx,yy, color = 'g',ls = "None", marker = "*")
#ax.set_aspect(1)    
#plt.show()

### Do analysis pixel by pixel and check if correlates withthe original ellipse

In [24]:
def circlediscretization(X_crns_L,Y_crns_L,xgrid_L,ygrid_L,cx0_L,cy0_L,r0_L,point,subpixel_grid,pixel_width):
       
    lstL = []
    ptchsL = []

    for i in range(X_crns_L.flatten().shape[0]):
        cx0max_L = X_crns_L.flatten()[i]
        cy0max_L = Y_crns_L.flatten()[i]

        # translate the left bottom corner
        x_pixel_L = cx0max_L+xgrid_L
        y_pixel_L = cy0max_L+ygrid_L
        # create sub-pixel grid
        xx_pixel_L, yy_pixel_L = np.meshgrid(x_pixel_L,y_pixel_L)
        
        circ_L = np.sqrt((xx_pixel_L-cx0_L[point,:])**2 + (yy_pixel_L-cy0_L[point,:])**2) <= r0_L[point,:]
        circ_corners_L = np.where(circ_L == True)
        xxgrid_L = np.array(circ_corners_L).T


        lstL.append((xxgrid_L.shape[0]/(subpixel_grid**2))*100)

        ptchsL.append(Rectangle((cx0max_L,cy0max_L), pixel_width,pixel_width))

    return xxgrid_L,lstL,ptchsL

In [28]:
def ellipsediscretization(X_crns_L,Y_crns_L,xgrid_L,ygrid_L,xc_arr_L,yc_arr_L,a_arr_L,b_arr_L,theta_arr,point,subpixel_grid,pixel_width):
    """
    Based on the parameters fed to the function, it discretizes the ellipse returning the occupied percentage of each pixel by the ellipse
    ----------
    X_crns: np.array
        Coordinate mesh grid on the x axis
    Y_crns:    np.array   
        Coordinate mesh grid on the y axis 
    x_subgrid:    np.array
        x-axis grid for the subgrid portion
    y_subgrid:    np.array
        y-axis grid for the subgrid portion
    xc_arr
    yc_arr
    a_arr
    b_arr
    theta_arr
    chess
    point
    subpixel_grid
    pixel_width
    
    Returns    
    -------    
    world_pts_arr : np.array
        world coordinates
    x_zhang_R:    np.array
        right image coordinates
    x_zhang_L:    np.array
        Left image coordinates
    P_L : np.array
        Left camera Projection matrix
    P_R : np.array
        Right camera Projection matrix
                
    """
    
    lstL = []
    ptchsL = []

    for i in range(X_crns_L.flatten().shape[0]):
        cx0max_L = X_crns_L.flatten()[i]
        cy0max_L = Y_crns_L.flatten()[i]

        # translate the left bottom corner
        x_pixel_L = cx0max_L+xgrid_L
        y_pixel_L = cy0max_L+ygrid_L
        # create sub-pixel grid
        xx_pixel_L, yy_pixel_L = np.meshgrid(x_pixel_L,y_pixel_L)

        x_L, y_L = xx_pixel_L.flatten(), yy_pixel_L.flatten()
                
        # calculate whether a point is inside the ellipse or not
        cos_angle = np.cos(np.radians(180.-theta_arr*180/np.pi))
        sin_angle = np.sin(np.radians(180.-theta_arr*180/np.pi))

        x_c_L = x_L - xc_arr_L
        y_c_L = y_L - yc_arr_L

        xct_L = x_c_L * cos_angle - y_c_L * sin_angle
        yct_L = x_c_L * sin_angle + y_c_L * cos_angle 

        rad_cc = (xct_L**2/(a_arr_L)**2) + (yct_L**2/(b_arr_L)**2)

        # set the 
        xxgrid_L = np.where(rad_cc <= 1.)[0]


        lstL.append((xxgrid_L.shape[0]/(subpixel_grid**2))*100)

        ptchsL.append(Rectangle((cx0max_L,cy0max_L), 1,1))
    
    return xxgrid_L,lstL,ptchsL 

In [29]:
coordinates_orig_L = (np.array([cx0_L,cy0_L]).T).reshape((zdistance.shape[0],2))
coordinates_orig_R = (np.array([cx0_R,cy0_R]).T).reshape((zdistance.shape[0],2))

In [30]:

from matplotlib.path import Path


import warnings
warnings.filterwarnings("error")

centroid_L_arr = np.zeros((x0_L.shape[0],x0_L.shape[1],2))
xgrid_L = np.linspace(0,1,subpixel_grid)
ygrid_L = np.linspace(0,1,subpixel_grid)

chess_pts_L = []
chess_ptchs_L = []

centroid_L_arr = np.zeros((x0_L.shape[0],2))
# iterate the chessboards


for point_i in range(x0_L.shape[0]):
    
    lstL = []
    ptchsL = []
    
    # meshgrid for the pixel
    [X_crns_L,Y_crns_L] = np.meshgrid(x_L_lst[point_i],y_L_lst[point_i])

    xxgrid_L,lstL,ptchsL = circlediscretization(X_crns_L,Y_crns_L,xgrid_L,ygrid_L,cx0_L,cy0_L,r0_L,point_i,subpixel_grid,1)
    # if the max and min values as 0 it means the point was not correctly identified
    if(np.max(lstL) == 0 and np.min(lstL) == 0):
        xxgrid_L,lstL,ptchsL = circlediscretization(X_crns_L,Y_crns_L,xgrid_L,ygrid_L,cx0_L,cy0_L,r0_L,point_i,subpixel_grid,1)
                                                    

    centroid_L_x = np.round(np.sum(np.multiply(X_crns_L.flatten(),lstL))/(np.sum(lstL)) + 0.5,decimals =0)
    centroid_L_y = np.round(np.sum(np.multiply(Y_crns_L.flatten(),lstL))/(np.sum(lstL)) + 0.5,decimals =0)
    centroid_L_arr[point_i,:]  = (centroid_L_x,centroid_L_y)

    chess_pts_L.append(lstL)
    chess_ptchs_L.append(ptchsL)            


In [27]:
centroid_L_arr

array([[1651.,  635.],
       [1651.,  635.],
       [1309.,  622.],
       [1196.,  617.],
       [1139.,  615.],
       [1105.,  613.],
       [1082.,  613.],
       [1066.,  612.],
       [1053.,  611.],
       [1044.,  611.],
       [1036.,  611.],
       [1030.,  610.],
       [1025.,  610.],
       [1021.,  610.],
       [1017.,  610.],
       [1014.,  610.],
       [1011.,  610.],
       [1008.,  610.],
       [1006.,  610.],
       [1004.,  609.],
       [1002.,  609.],
       [1001.,  609.],
       [ 999.,  609.],
       [ 998.,  609.],
       [ 996.,  609.],
       [ 995.,  609.],
       [ 994.,  609.],
       [ 993.,  609.],
       [ 992.,  609.],
       [ 992.,  609.],
       [ 991.,  609.],
       [ 990.,  609.],
       [ 989.,  609.],
       [ 989.,  609.],
       [ 988.,  609.],
       [ 988.,  609.],
       [ 987.,  609.],
       [ 986.,  609.],
       [ 986.,  609.],
       [ 986.,  609.],
       [ 985.,  609.],
       [ 985.,  609.],
       [ 984.,  609.],
       [ 98

In [34]:

warnings.filterwarnings("error")

centroid_R_arr = np.zeros((x0_R.shape[0],x0_R.shape[1],2))
xgrid_R = np.linspace(0,1,subpixel_grid)
ygrid_R = np.linspace(0,1,subpixel_grid)

chess_pts_R = []
chess_ptchs_R = []

centroid_R_arr = np.zeros((x0_R.shape[0],2))
# iterate the chessboards


for point_i in range(x0_R.shape[0]):
    
    lstR = []
    ptchsR = []
    
    # meshgrid for the pixel
    [X_crns_R,Y_crns_R] = np.meshgrid(x_R_lst[point_i],y_R_lst[point_i])


    xxgrid_R,lstR,ptchsR = circlediscretization(X_crns_R,Y_crns_R,xgrid_R,ygrid_R,cx0_R,cy0_R,r0_R,point_i,subpixel_grid,1)

    # if the max and min values as 0 it means the point was not correctly identified
    if(np.max(lstR) == 0 and np.min(lstR) == 0):
        xxgrid_R,lstR,ptchsR = circlediscretization(X_crns_R,Y_crns_R,xgrid_R,ygrid_R,cx0_R,cy0_R,r0_R,point_i,subpixel_grid,1)
                                                    

    centroid_R_x = np.round(np.sum(np.multiply(X_crns_R.flatten(),lstR))/(np.sum(lstR)) + 0.5,decimals = 0)
    centroid_R_y = np.round(np.sum(np.multiply(Y_crns_R.flatten(),lstR))/(np.sum(lstR)) + 0.5,decimals = 0)
    centroid_R_arr[point_i,:]  = (centroid_R_x,centroid_R_y)

    chess_pts_R.append(lstR)
    chess_ptchs_R.append(ptchsR)            


In [35]:
centroid_R_arr

array([[285., 635.],
       [285., 635.],
       [627., 622.],
       [740., 617.],
       [797., 615.],
       [831., 613.],
       [854., 613.],
       [870., 612.],
       [883., 611.],
       [892., 611.],
       [900., 611.],
       [906., 610.],
       [911., 610.],
       [915., 610.],
       [919., 610.],
       [922., 610.],
       [925., 610.],
       [928., 610.],
       [930., 610.],
       [932., 609.],
       [934., 609.],
       [935., 609.],
       [937., 609.],
       [938., 609.],
       [940., 609.],
       [941., 609.],
       [942., 609.],
       [943., 609.],
       [944., 609.],
       [944., 609.],
       [945., 609.],
       [946., 609.],
       [947., 609.],
       [947., 609.],
       [948., 609.],
       [948., 609.],
       [949., 609.],
       [950., 609.],
       [950., 609.],
       [950., 609.],
       [951., 609.],
       [951., 609.],
       [952., 609.],
       [952., 609.],
       [952., 609.],
       [953., 609.],
       [953., 609.],
       [953.,

In [36]:
np.sum(centroid_R_arr[:,:] - (coordinates_orig_R[:,:]))

-22.76202392578125

# Matrixes

In [37]:
P_L_original_p = [np.array([[2.73037543e+03, 0.00000000e+00, 9.68000000e+02, 0.00000000e+00],
                        [0.00000000e+00, 2.73037543e+03, 6.08000000e+02, 0.00000000e+00],
                        [0.00000000e+00, 0.00000000e+00, 1.00000000e+00, 0.00000000e+00]]),
                np.array([[2.73037543e+03, 0.00000000e+00, 9.68000000e+02, 0.00000000e+00],
                        [0.00000000e+00, 2.73037543e+03, 6.08000000e+02, 0.00000000e+00],
                        [0.00000000e+00, 0.00000000e+00, 1.00000000e+00, 0.00000000e+00]]),
                np.array([[2.73037543e+03, 0.00000000e+00, 9.68000000e+02, 0.00000000e+00],
                        [0.00000000e+00, 2.73037543e+03, 6.08000000e+02, 0.00000000e+00],
                        [0.00000000e+00, 0.00000000e+00, 1.00000000e+00, 0.00000000e+00]]),
                np.array([[2.73037543e+03, 0.00000000e+00, 9.68000000e+02, 0.00000000e+00],
                        [0.00000000e+00, 2.73037543e+03, 6.08000000e+02, 0.00000000e+00],
                        [0.00000000e+00, 0.00000000e+00, 1.00000000e+00, 0.00000000e+00]]),
                np.array([[2.73037543e+03, 0.00000000e+00, 9.68000000e+02, 0.00000000e+00],
                        [0.00000000e+00, 2.73037543e+03, 6.08000000e+02, 0.00000000e+00],
                        [0.00000000e+00, 0.00000000e+00, 1.00000000e+00, 0.00000000e+00]]),
                np.array([[2.73037543e+03, 0.00000000e+00, 9.68000000e+02, 0.00000000e+00],
                        [0.00000000e+00, 2.73037543e+03, 6.08000000e+02, 0.00000000e+00],
                        [0.00000000e+00, 0.00000000e+00, 1.00000000e+00, 0.00000000e+00]])]                        

In [38]:
P_R_original_p =  [np.array([[ 2.73037543e+03,  0.00000000e+00,9.68000000e+02,-1.36518771e+06],
                        [ 0.00000000e+00,  2.73037543e+03,  6.08000000e+02,0.00000000e+00],
                        [ 0.00000000e+00,  0.00000000e+00,  1.00000000e+00,0.00000000e+00]]),
                np.array([[ 2.73037543e+03,  0.00000000e+00,  9.68000000e+02,-1.36518771e+06],
                        [ 0.00000000e+00,  2.73037543e+03,  6.08000000e+02,0.00000000e+00],
                        [ 0.00000000e+00,  0.00000000e+00,  1.00000000e+00,0.00000000e+00]]),
                np.array([[ 2.73037543e+03,  0.00000000e+00,  9.68000000e+02,-1.36518771e+06],
                        [ 0.00000000e+00,  2.73037543e+03,  6.08000000e+02,0.00000000e+00],
                        [ 0.00000000e+00,  0.00000000e+00,  1.00000000e+00,0.00000000e+00]]),
                np.array([[ 2.73037543e+03,  0.00000000e+00,  9.68000000e+02,-1.36518771e+06],
                        [ 0.00000000e+00,  2.73037543e+03,  6.08000000e+02,0.00000000e+00],
                        [ 0.00000000e+00,  0.00000000e+00,  1.00000000e+00,0.00000000e+00]]),
                np.array([[ 2.73037543e+03,  0.00000000e+00,  9.68000000e+02,-1.36518771e+06],
                        [ 0.00000000e+00,  2.73037543e+03,  6.08000000e+02,0.00000000e+00],
                        [ 0.00000000e+00,  0.00000000e+00,  1.00000000e+00,0.00000000e+00]]),
                np.array([[ 2.73037543e+03,  0.00000000e+00,  9.68000000e+02,-1.36518771e+06],
                        [ 0.00000000e+00,  2.73037543e+03,  6.08000000e+02,0.00000000e+00],
                        [ 0.00000000e+00,  0.00000000e+00,  1.00000000e+00,0.00000000e+00]])]


In [39]:
ret_S_2_p = [[0.2637375195395553],
             [0.5208904811475614],
             [0.39595311339867384],
             [0.4774205929812345],
             [0.5403807164757266],
             [0.5129051705850326]]         

In [40]:
new_mtxL_list_2_p =     [[np.array([[3177.394344, 0.000000, 868.514520],
                                [0.000000, 3193.865542, 613.104839],
                                [0.000000, 0.000000, 1.000000]])],
                        [np.array([[2601.802800, 0.000000, 913.720774],
                                [0.000000, 2603.661860, 640.282020],
                                [0.000000, 0.000000, 1.000000]])],
                        [np.array([[2590.965467, 0.000000, 907.098782],
                                [0.000000, 2595.823358, 621.646273],
                                [0.000000, 0.000000, 1.000000]])],
                        [np.array([[2619.382584, 0.000000, 908.967047],
                                [0.000000, 2622.730112, 606.658303],
                                [0.000000, 0.000000, 1.000000]])],
                        [np.array([[2585.978443, 0.000000, 942.514399],
                                [0.000000, 2588.592868, 594.521434],
                                [0.000000, 0.000000, 1.000000]])],
                        [np.array([[2574.733760, 0.000000, 946.084130],
                                [0.000000, 2577.776040, 590.873339],
                                [0.000000, 0.000000, 1.000000]])]] 

In [41]:
ret_L_list_2_p =    [[0.17196515935514858],
                    [0.47863845277104067],
                    [0.36123020670433137],
                    [0.44011645068186067],
                    [0.4872057983230428],
                    [0.4672643081187851]]

In [42]:
new_mtxR_list_2_p =     [[np.array([[2740.523940, 0.000000, 1011.343989],
                                [0.000000, 2745.916176, 673.805539],
                                [0.000000, 0.000000, 1.000000]])],
                        [np.array([[2605.435065, 0.000000, 1000.655553],
                                [0.000000, 2607.384771, 677.607012],
                                [0.000000, 0.000000, 1.000000]])],
                        [np.array([[2606.595357, 0.000000, 989.407340],
                                [0.000000, 2609.168364, 657.813164],
                                [0.000000, 0.000000, 1.000000]])],
                        [np.array([[2641.709842, 0.000000, 995.427222],
                                [0.000000, 2646.217062, 635.580703],
                                [0.000000, 0.000000, 1.000000]])],
                        [np.array([[2579.183470, 0.000000, 999.864075],
                                [0.000000, 2582.098697, 627.927045],
                                [0.000000, 0.000000, 1.000000]])],
                        [np.array([[2568.956315, 0.000000, 1000.924975],
                                [0.000000, 2572.199945, 625.579566],
                                [0.000000, 0.000000, 1.000000]])]]                                  

In [43]:
ret_R_list_2_p =    [[0.2156546707277009],      
                    [0.5060286563587989],
                    [0.3797317627153738],
                    [0.47441006626612464],
                    [0.5218551163958846],
                    [0.4851207604596431]]                                    

In [44]:


Rot_list_2_p =  [[np.array([[0.998009, 0.001370, -0.063064],
                            [-0.001795, 0.999976, -0.006681],
                            [0.063053, 0.006781, 0.997987]])],
                [np.array([[0.998257, 0.002625, -0.058951],
                            [-0.002297, 0.999982, 0.005621],
                            [0.058965, -0.005476, 0.998245]])],
                [np.array([[0.998206, 0.003594, -0.059765],
                            [-0.003167, 0.999969, 0.007237],
                            [0.059790, -0.007035, 0.998186]])],
                [np.array([[0.998325, 0.003811, -0.057737],
                            [-0.003302, 0.999955, 0.008916],
                            [0.057769, -0.008710, 0.998292]])],
                [np.array([[0.998593, 0.004110, -0.052868],
                            [-0.003729, 0.999966, 0.007303],
                            [0.052897, -0.007095, 0.998575]])],
                [np.array([[0.998581, 0.004217, -0.053089],
                            [-0.003844, 0.999967, 0.007113],
                            [0.053117, -0.006899, 0.998564]])]]

In [45]:
Trns_list_2_p =  [[np.array([[-486.148518],
                            [7.986519],
                            [-349.481345]])],
                 [np.array([[-480.150365],
                            [-1.032461],
                            [-18.354234]])],
                 [np.array([[-475.604731],
                            [-2.995716],
                            [-9.700418]])],
                 [np.array([[-481.395487],
                            [-1.101631],
                            [-1.375279]])],
               [np.array([[-471.054816],
                            [-1.084550],
                            [-21.705676]])],
               [np.array([[-469.311981],
                            [-1.587343],
                            [-21.457134]])]]

In [46]:
P_L_2_list_p = []
P_R_2_list_p = []

for i in range(len(ret_R_list_2_p)):
    R1,R2,P_L_est,P_R_est,Q,roi_left,roi_right = cv2.stereoRectify(new_mtxL_list_2_p[i][0], ret_L_list_2_p[i][0], new_mtxR_list_2_p[i][0], ret_R_list_2_p[i][0], (image_size[0],image_size[1]), Rot_list_2_p[i][0], Trns_list_2_p[i][0],flags = cv2.CALIB_ZERO_DISPARITY)
    P_L_2_list_p.append(P_L_est)
    P_R_2_list_p.append(P_R_est)


In [47]:
x_arr_L, X_arr_L, E_L, K_L,P_L = get_image_points(X,PX,PY,thetax=THETA_X_L,thetay = THETA_Y_L,thetaz = THETA_Z_L,trans_x= -C_L[0],trans_y= -C_L[1],trans_z= -C_L[2],F = F,mx = (1/pixel_width),my =(1/pixel_width))
x_arr_R, X_arr_R, E_R, K_R,P_R = get_image_points(X,PX,PY,thetax = THETA_X_R,thetay = THETA_Y_R, thetaz = THETA_Z_R,trans_x= -C_R[0],trans_y= -C_R[1],trans_z= -C_R[2],F = F,mx = (1/pixel_width),my =(1/pixel_width))

In [48]:
n_distance = len(zdistance)
len_chess = len(ret_L_list_2_p)

XestOpenCV_2 = np.zeros((len_chess,n_distance,4,1))
Xoriginal_2 = np.zeros((len_chess,n_distance,4,1))

estError_2 = np.zeros((len_chess,n_distance,1))

Error_2_abs_Z = np.zeros((len_chess,n_distance,1)) 
relative_Error_2_Z = np.zeros((len_chess,n_distance,1)) 

X_est_arr_2 = np.zeros((len_chess,n_distance,4,1))
X_org_arr_2 = np.zeros((len_chess,n_distance,4,1))

triangulation_error_2_p =np.zeros((len_chess,1))



for i in range(len(ret_R_list_2_p)):
    for j in range(len(centroid_L_arr)):
        # 2 meters
        XestOpenCV_2[i,j,:,:] = cv2.triangulatePoints(P_L_2_list_p[i],P_R_2_list_p[i], centroid_L_arr[j], centroid_R_arr[j])
        Xoriginal_2[i,j,:,:] =  cv2.triangulatePoints(P_L_original_p[i],P_R_original_p[i],x_arr_L[j], x_arr_R[j])

        X_est_arr_2[i,j,:,:] = XestOpenCV_2[i,j,:,:]/XestOpenCV_2[i,j,-1,:]
        X_org_arr_2[i,j,:,:] = Xoriginal_2[i,j,:,:]/Xoriginal_2[i,j,-1,:]

        Error_2_abs_Z[i,j,:] = np.abs(X_est_arr_2[i,j,2,0]-X_org_arr_2[i,j,2,0])
        relative_Error_2_Z[i,j,:] = np.abs(X_est_arr_2[i,j,2,0]-X_org_arr_2[i,j,2,0])/X_org_arr_2[i,j,2,0]*100
        

        estError_2[i,j,:] = np.sqrt(np.sum(np.square(X_est_arr_2[i,j,:3,:]-X_org_arr_2[i,j,:3,:])))


    triangulation_error_2_p[i,:] = np.sum(estError_2[i,:,:])/estError_2[i,:,:].shape[0]



In [49]:
np.save("E_err2_p_centroid_noinfi_round.npy",Error_2_abs_Z)
np.save("relative_Error_2_Z_centroid_noinfi_round.npy",relative_Error_2_Z)
np.save("X_org_arr_2_centroid_noinfi.npy",X_org_arr_2)
np.save("X_est_arr_2_centroid_noinfi.npy",X_est_arr_2)

In [52]:
#estError_20_distance
E_err2_p = np.zeros((estError_2.shape[0],n_distance))

for j in range(estError_2.shape[0]):
    for i in range(estError_2.shape[1]):
        E_err2_p[j,i] = np.sum(np.abs(X_est_arr_2[j,i,:3,:]-X_org_arr_2[j,i,:3,:]))/(np.sum(X_est_arr_2[j,i,:3,:]))*100


In [53]:
fig = plt.figure(figsize = (20,10))
ax = fig.add_subplot()

x_marker = np.linspace(10e-10,1e3,1000)
y_marker = np.ones((1000,1))
#ax.plot(x_marker,y_marker*0.001,'b--')

color_2 = ['black','firebrick','gold','chartreuse','darkcyan','m']
sub_2 = ["2 meters with 5 chessboards for no infi","2 meters with 20 chessboards for no infi","2 meters with 45 chessboards for no infi","2 meters with 80 chessboards for no infi","2 meters with 125 chessboards for no infi","2 meters with 180 chessboards for no infi"]

for i in range(estError_2.shape[0]):

    ax.plot(zdistance/1000,E_err2_p[i,:],color =  color_2[i], label = sub_2[i], marker = '^')


ax.set_yscale('log')


ax.set_xlim(0,50)
ax.set_ylabel("Triangulation errror [%]")
ax.set_title("Triangulation error")
ax.set_xlabel("Distance [m]")
ax.legend()
ax.grid()

In [45]:
# number of chessboards
nr_chess = [5,20,45,80,125,180]


fig = plt.figure(figsize = (20,10))
ax = fig.add_subplot(211)
ax2 = fig.add_subplot(212)

x_marker = np.linspace(10e-10,1e3,1000)
y_marker = np.ones((1000,1))
#ax.plot(x_marker,y_marker*0.001,'b--')

color_2 = ['black','firebrick','gold','chartreuse','darkcyan','m']
sub_2 = ["2 meters with 5 chessboards w/ centroding","2 meters with 20 chessboards w/ centroding","2 meters with 45 chessboards w/ centroding","2 meters with 80 chessboards w/ centroding","2 meters with 125 chessboards w/ centroding","2 meters with 180 chessboards w/ centroding"]
color_3 = ['red','g','orange','c','darkblue','fuchsia']
sub_3 = ["3 meters with 5 chessboards w/ centroding","3 meters with 20 chessboards w/ centroding","3 meters with 45 chessboards w/ centroding","3 meters with 80 chessboards w/ centroding","3 meters with 125 chessboards w/ centroding","3 meters with 180 chessboards w/ centroding"]
color_4 = ['darkslategray','brown','tomato','aqua','lime','y']
sub_4 = ["4 meters with 5 chessboards w/ centroding","4 meters with 20 chessboards w/ centroding","4 meters with 45 chessboards w/ centroding","4 meters with 80 chessboards w/ centroding","4 meters with 125 chessboards w/ centroding","4 meters with 180 chessboards w/ centroding"]

for i in range(X_org_arr_2.shape[0]):

    ax.plot(zdistance[:50]/1000,X_est_arr_2[i,:50,2]/1000,color =  color_2[i] , marker = '^', label = sub_2[i])
    ax.plot(zdistance[:50]/1000,zdistance[:50]/1000,color = 'b',label = "1:1 correspondence", marker = '*')

ax2.plot(nr_chess,triangulation_error_2_p/1000,color = 'b'  , marker = '^',linestyle = 'dashed', label = sub_2[i])
ax2.plot(nr_chess,triangulation_error_3_p/1000,color =  'g' , marker = '^',linestyle = 'dashed', label = sub_3[i])
ax2.plot(nr_chess,triangulation_error_4_p/1000,color =  'k' , marker = '^',linestyle = 'dashed', label = sub_4[i])

    
ax2.set_yscale('log')


ax.set_ylabel("Triangulation distance[m] ")
ax.set_title("Triangulation distance")
ax.set_xlabel("Distance [m]")
#ax.legend()
ax.grid()
ax2.grid()